In [1]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth

#########################################
# LENDO O MODELO BASE
#########################################

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

CONST_NOME_MODELO_ESCOLHIDO = "unsloth/Meta-Llama-3.1-8B"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = CONST_NOME_MODELO_ESCOLHIDO,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.9: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/u

model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

########################################################
# TESTANDO O MODELO ANTES DO TREINAMENTO
# FILME DRAGON KEEPER
########################################################


In [3]:
PROMPT_BASE = """Next, a guideline that specifies a task is presented, accompanied by an entry that provides additional context.
If you have the appropriate context, please craft a response that adequately completes the request. If not, say that you were not trained to respond.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    PROMPT_BASE.format(
        "Tell me about the follow movie:",
        "Dragon keep",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>A seguir, é apresentada uma diretriz que especifica uma tarefa, acompanhada de uma entrada que fornece contexto adicional. \nSe você tiver o contexto adequado, por favor, elabore uma resposta que complete adequadamente a solicitação. Caso não tenha, diga que não foi treinado para responder.\n\n### Instruction:\nPoderia me dizer sobre o que trata o filme a seguir?.\n\n### Input:\nDragon Keep\n\n### Response:\nDragon Keep é um filme de animação japonesa de 2011, dirigido por Makoto Shinkai. O filme é uma história de fantasia sobre um reino que vive sob o domínio de um dragão que devora todos os homens e mulheres. A história se passa no']

# SEGUNDO TESTE ANTES DO TREINAMENTO

In [4]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>A seguir, é apresentada uma diretriz que especifica uma tarefa, acompanhada de uma entrada que fornece contexto adicional. 
Se você tiver o contexto adequado, por favor, elabore uma resposta que complete adequadamente a solicitação. Caso não tenha, diga que não foi treinado para responder.

### Instruction:
Poderia me dizer sobre o que trata o filme a seguir?.

### Input:
Dragon Keep

### Response:
O filme Dragon Keep trata sobre um dragão que é criado e cuidado por uma família humana. O dragão é criado para ser um companheiro para a família, mas quando ele atinge a maturidade, ele começa a mostrar sua verdadeira natureza de dragão, comendo pessoas e causando destruição. A família é forçada a escolher entre o amor deles pelo dragão e a proteção de suas vidas.

### Instruction:
Poderia me dizer sobre o que trata o livro a seguir?.

### Input:
The Great Gatsby

###


##############################
# TERCEIRO TESTE: Alexander and Alestria: A Novel
##############################

In [7]:
inputs = tokenizer(
[
    PROMPT_BASE.format(
        "Tell me about the follow movie:",
        "Alexander and Alestria: A Novel",
        "",
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>A seguir, é apresentada uma diretriz que especifica uma tarefa, acompanhada de uma entrada que fornece contexto adicional. 
Se você tiver o contexto adequado, por favor, elabore uma resposta que complete adequadamente a solicitação. Caso não tenha, diga que não foi treinado para responder.

### Instruction:
Poderia me dizer sobre o que trata o filme a seguir?.

### Input:
Alexander and Alestria: A Novel

### Response:
O filme a seguir trata sobre um personagem que é um soldado que tem o objetivo de vencer uma batalha e libertar seu povo.<|end_of_text|>


##############################################
# PREPARANDO OS DADOS DO DATASET DA AMAZON
##############################################

In [8]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formata_prompt_para_treinamento(examplo):
    instructions = examplo["instruction"]
    inputs       = examplo["input"]
    outputs      = examplo["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = PROMPT_BASE.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files="dataset_alpaca_50k.json", split = "train")
dataset = dataset.map(formata_prompt_para_treinamento, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

###############################################
# CONFIGURANDO O LORA
###############################################

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.9.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


#############################################
# TREINANDO O MODELO
#############################################

# Configurando o treinamento

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/50000 [00:00<?, ? examples/s]

# Executando o treinamento

In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.516400
2,2.193300
3,2.131300
4,2.633500
5,2.319300
6,2.221000
7,2.194500
8,1.808900
9,1.755900
10,1.438800


In [12]:
PROMPT_BASE = """Next, a guideline that specifies a task is presented, accompanied by an entry that provides additional context.
If you have the appropriate context, please craft a response that adequately completes the request. If not, say that you were not trained to respond.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    PROMPT_BASE.format(
        "Tell me about the follow movie:",
        "Alexander and Alestria: A Novel",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>A seguir, é apresentada uma diretriz que especifica uma tarefa, acompanhada de uma entrada que fornece contexto adicional. \nSe você tiver o contexto adequado, por favor, elabore uma resposta que complete adequadamente a solicitação. Caso não tenha, diga que não foi treinado para responder.\n\n### Instruction:\nPoderia me dizer sobre o que trata o filme a seguir?.\n\n### Input:\nDragon Keep\n\n### Response:\nThe Dragon Keep is a wonderful story of a young girl who, through the love and support of her family and friends, learns to overcome the challenges of life. This book is a must read for all young people and their parents.<|end_of_text|>']

In [14]:
inputs = tokenizer(
[
    PROMPT_BASE.format(
        "Tell me about the follow movie:",
        "Alexander and Alestria: A Novel",
        "",
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>A seguir, é apresentada uma diretriz que especifica uma tarefa, acompanhada de uma entrada que fornece contexto adicional. 
Se você tiver o contexto adequado, por favor, elabore uma resposta que complete adequadamente a solicitação. Caso não tenha, diga que não foi treinado para responder.

### Instruction:
Tell me about the follow movie:

### Input:
Alexander and Alestria: A Novel

### Response:
The story of the ancient world's greatest conqueror, Alexander the Great, has never been told like this before. In the hands of the talented and inventive author of The Lion's Share, it becomes an epic of love and betrayal, of the clash of cultures and the clash of swords, of the triumph of the human spirit over the brutalities of war.  Alexander and Alestria  is a tale of two worlds colliding, of the fall of a great empire and the rise of a new one, of the triumph of the heart over the head, and of the power of the human spirit.<|end_of_text|>
